In [1]:
import csv
import json
import re
import ast

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def getItem(category):
    if category is None:
        url = 'https://api.nookplaza.net/items'
    else:
        url = 'https://api.nookplaza.net/items?category={}'.format(category)
    r = requests.get(url)
    text = r.json()
    print('get {} {}'.format(category, len(text['results'])))
    return text['results']

In [3]:
def saveFile(category, output, dictName):
    with open('../../nook_link/src/data/{}.json'.format(category), 'wt') as fout:
        fout.write(json.dumps(output))

    with open('{}.json'.format(category), 'wt') as fout:
        fout.write(json.dumps(output))

    with open('{}.csv'.format(category), 'wt', encoding='utf-8', newline='') as fout:
        csvout = csv.writer(fout)

        output_csv = [dictName]
        for row in output:
            row_ = [row[name] for name in dictName]
            output_csv.append(row_)
        csvout.writerows(output_csv)
    
    print('save {} over'.format(category))

In [4]:
def getEn2twFn():
    nookplazaUrl = 'https://nookplaza.net'
    r = requests.get(nookplazaUrl)

    m = re.search('main\.dart\.js\?v=(\d*)', r.text)
    dartJs = m.group()

    mainJsUrl = '{}/{}'.format(nookplazaUrl, dartJs)
    r = requests.get(mainJsUrl)
    mainJs = r.text

    # 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
    m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
    m = [item.encode("utf-8").decode('unicode-escape') for item in m]
    m = [ast.literal_eval(item) for item in m if not ':null' in item]

    nameMap = {}
    for item in m:
        nameMap[item["en-us"]] = item

    print('get {} i18n data'.format(len(nameMap.keys())))

    try:
        df = pd.read_excel('../tempData/Furniture.xlsx', sheetname=0)
    except:
        df = pd.read_excel('../tempData/Furniture.xlsx', sheet_name=0)

    EnTwTable = ['類別', '取得方式', '標籤', '主題', '款式', '樣式', '款式說明', '樣式說明', 'DIY相關', '藝術品', '作者/年代/材質', '藝術品真實圖', '村民']

    nameMapCustom = {}
    for EnTw in EnTwTable:
        itemType_en = df[EnTw].tolist()
        itemType_tw = df['{}_tw'.format(EnTw)].tolist()
        itemType_en = [x for x in itemType_en if str(x) != 'nan']
        itemType_tw = [x for x in itemType_tw if str(x) != 'nan']

        for item in zip(itemType_en, itemType_tw):
            if item[0] in nameMapCustom and item[1] != nameMapCustom[item[0]]['zh-tw']:
                print(item)
                print(nameMapCustom[item[0]])
                print('------\n')

            nameMapCustom[item[0]] = {
                'zh-tw': item[1]
            }

    print('Custom {} i18n data'.format(len(nameMapCustom.keys())))

    def en2twFn(en, Custom = False):
        if Custom == 'onlySite':
            return nameMap[en]

        if Custom:
            if en in nameMapCustom:
                return nameMapCustom[en]

            elif en.lower() in nameMapCustom:
                return nameMapCustom[en.lower()]

            elif (en[0].lower() + en[1:]) in nameMapCustom:
                return nameMapCustom[(en[0].lower() + en[1:])]

        if en.lower() in nameMap:
            return nameMap[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMap:
            return nameMap[(en[0].lower() + en[1:])]

        elif en in nameMap:
            return nameMap[en]

        elif en in nameMapCustom:
                return nameMapCustom[en]

        elif en.lower() in nameMapCustom:
            return nameMapCustom[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMapCustom:
            return nameMapCustom[(en[0].lower() + en[1:])]

        elif en.replace('The', 'the') in nameMap:
            return nameMap[en.replace('The', 'the')]

        elif en.replace('K.K. Tude', 'K.K. Étude') in nameMap:
            return nameMap[en.replace('K.K. Tude', 'K.K. Étude')]

        elif en.replace('Caf K.K.', 'Café K.K.') in nameMap:
            return nameMap[en.replace('Caf K.K.', 'Café K.K.')]

        print(en)
        raise en
        
    return en2twFn

In [5]:
en2twFn = getEn2twFn()

idMap = {}
name2fileNameMap = {}
allItem = getItem(None)
for item in allItem:
#     if 'ifier' in item['name']:
#         print(item)
        
    if item['name'].startswith("Present ("):
        continue
        
    if 'internalID' in item['content']:
        try:
            name = en2twFn(item['name'].replace(' (fake)', ''))['zh-tw']
            idMap[item['content']['internalID']] = {'name': name, 'filename': item['content']['image']}
            name2fileNameMap[item['name']] = item['content']['filename']
        except:
            pass
#             print(item['name'])

print('get {} id to Name data'.format(len(idMap.keys())))

get 6047 i18n data
Custom 1199 i18n data
get None 5845
Bell bag
Turnips
Coin
get 4589 id to Name data


In [10]:
en2twFn('Pacifier')

# nameMap['Pacifier']
# name2fileNameMap['Stormy-night wall']

{'en-us': 'pacifier',
 'en-gb': 'pacifier',
 'de-de': 'Schnullers',
 'es-es': 'chupete',
 'fr-fr': 'tétine',
 'fr-ca': 'suce',
 'it-it': 'ciuccio',
 'nl-nl': 'speen',
 'zh-cn': '奶嘴',
 'zh-tw': '奶嘴',
 'ja-jp': 'おしゃぶり',
 'ko-kr': '노리개젖꼭지',
 'ru-ru': 'соска-пустышка'}

In [6]:
en2twFn = getEn2twFn()
output = []
furnitures = getItem('Furniture')
recipes = getItem('Recipes')

DIYMap = {}
for diy in recipes:
    DIYMap[diy['name']] = diy['content']

for item in furnitures:
    if item["category"] == "Art":
        continue

    pickData = {'type': 'furnitures'}
    name = en2twFn(item['name'])
    
#     print(name['zh-tw'])
    
    pickData['id'] = int(item['content']['internalID'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']

    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else None
    pickData['sell'] = item['content']['sell']

    pickData['category'] = en2twFn(item['content']['category'], True)['zh-tw']
    pickData['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickData['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    
    pickData['interact'] = item['content']['interact']
    pickData['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['DIY'] = item['content']['dIY']
#     pickData['diyInfo'] = {}
    pickData['diyInfoMaterials'] = []
    pickData['diyInfoObtainedFrom'] = []
    pickData['diyInfoSourceNotes'] = ''

    if item['content']['dIY']:
        diyInfo = DIYMap[item['name']]
        pickData['diyInfoMaterials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in diyInfo['materials']]
        pickData['diyInfoObtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in diyInfo['obtainedFrom']]
        
        try:
            pickData['diyInfoSourceNotes'] = '' if diyInfo['sourceNotes'] == '' else en2twFn(diyInfo['sourceNotes'], True)['zh-tw']
        except:
            pickData['diyInfoSourceNotes'] = diyInfo['sourceNotes']
        
    pickData['filename'] = item['content']['filename']
    
    if 'bodyTitle' in item['content'] and item['content']['bodyTitle']:
        pickData['bodyTitle'] = en2twFn(item['content']['bodyTitle'], True)['zh-tw']
    else:
        pickData['bodyTitle'] = '無'
        
    if 'bodyCustomize' in item['content']:
        pickData['bodyCustomize'] = item['content']['bodyCustomize']
    else:
        pickData['bodyCustomize'] = False
    
    if 'patternTitle' in item['content'] and item['content']['patternTitle']:
        pickData['patternTitle'] = en2twFn(item['content']['patternTitle'], True)['zh-tw']
    else:
        pickData['patternTitle'] = '無'
        
    if 'patternCustomize' in item['content']:
        pickData['patternCustomize'] = item['content']['patternCustomize']
    else:
        pickData['patternCustomize'] = False

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        bodyIndex, pattrenIndex = content['filename'].split('_')[-2:]
        
        bodyColor = en2twFn(content['bodyColor'], True)['zh-tw'] if 'bodyColor' in content and content['bodyColor'] is not None else ''
        patternColor = en2twFn(content['patternColor'], True)['zh-tw'] if 'patternColor' in content and content['patternColor'] is not None else ''
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
    pickData['variations'] = variations
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY', 'diyInfoMaterials', 'diyInfoObtainedFrom', 'diyInfoSourceNotes']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'variations']

saveFile('furniture', output, dictName)

get 6047 i18n data
Custom 1199 i18n data
get Furniture 1131
get Recipes 609
Obtained during the main storyline during the villager house development quest if the native fruit is oranges
50% chance of being obtained during the main storyline during the villager house development quest
50% chance of being obtained during the main storyline during the villager house development quest
Fish up 3 empty cans
Obtained during the main storyline during the villager house development quest if the native fruit is peaches
Purchase Test Your DIY Skills from Nook's Cranny
Obtained during the main storyline during the villager house development quest if the native fruit is pears
50% chance of being obtained during the main storyline during the villager house development quest
Purchase Test Your DIY Skills from Nook's Cranny
Obtained during the main storyline during the villager house development quest if the native fruit is pears
Purchase Test Your DIY Skills from Nook's Cranny; Obtained during the ma

In [8]:
furnitures = getItem('Furniture')

ArtMap = {}
for item in furnitures:
    if item["category"] != "Art":
        continue
        
    pickArt = {'type': 'arts'}
    name = en2twFn(item['name'].replace(' (fake)', ''))

    pickArt['id'] = int(item['content']['internalID'])
    pickArt['category'] = en2twFn(item['category'], True)['zh-tw']
    pickArt['name_c'] = name['zh-tw']
    pickArt['name_j'] = name['ja-jp']
    pickArt['name_e'] = name['en-us']
    
    pickArt['artist'] = en2twFn(item['content']['artist'])['zh-tw']
    pickArt['filename'] = item['content']['filename']
    pickArt['interact'] = item['content']['interact']
    pickArt['itemCategory'] = item['content']['itemCategory']
    pickArt['museumDescription'] = item['content']['museumDescription']
    pickArt['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickArt['realArtworkTitle'] = item['content']['realArtworkTitle']
    pickArt['realArtworkTitle_tw'] = en2twFn(item['content']['realArtworkTitle'])['zh-tw']
    pickArt['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickArt['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    pickArt['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickArt['realArtImage'] = en2twFn(item['content']['filename'].replace('Fake', ''))['zh-tw']
    
    if not name['en-us'] in ArtMap:
        ArtMap[name['en-us']] = {}
        
    ArtMap[name['en-us']][str(item['content']['genuine'])] = pickArt
    
artOutput = []
for name in ArtMap.keys():
    if 'False' in ArtMap[name]:
        ArtMap[name]['True']['haveFake'] = True
    else:
        ArtMap[name]['True']['haveFake'] = False
    artOutput.append(ArtMap[name]['True'])

artOutput = sorted(artOutput, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'realArtworkTitle', 'realArtworkTitle_tw', 'artist', 'museumDescription']
dictName += ['obtainedFrom', 'filename', 'haveFake', 'interact', 'itemCategory', 'themes', 'tag', 'size', 'realArtImage']

saveFile('art', artOutput, dictName)

get Furniture 1131
save art over


In [7]:
output = []

recipes = getItem('Recipes')
allItem = getItem(None)

sellMap = {}
for item in allItem:
    if 'sell' in item['content'] and item['category'] != 'Recipes':
        sellMap[item['name']] = item['content']['sell']

for item in recipes:

#     print(item['name'])
#     print('')

    if 'construction kit' in item['name']:
        continue
    pickData = {'type': 'recipes'}
    name = en2twFn(item['name'])

    pickData['id'] = int(item['content']['internalID'])
#     print(name['zh-tw'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['category'] = en2twFn(item['content']['itemCategory'], True)['zh-tw']
    pickData['filename'] = item['content']['itemImage'].split('/')[-1].replace('.png', '')
    pickData['DIY'] = True
#     pickData['sell'] = item['content']['sell']
    pickData['sell'] = sellMap[item['name']]
    pickData['materials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in item['content']['materials']]
    pickData['obtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['obtainedFrom']]
#     pickData['sourceNotes'] = '' if item['content']['sourceNotes'] == '' else en2twFn(item['content']['sourceNotes'], True)['zh-tw']
    
    try:
        pickData['sourceNotes'] = '' if item['content']['sourceNotes'] == '' else en2twFn(item['content']['sourceNotes'], True)['zh-tw']
    except:
        pickData['sourceNotes'] = item['content']['sourceNotes']
    
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])
dictName = ['id', 'name_e', 'name_j', 'name_c', 'category', 'filename', 'sell', 'materials', 'obtainedFrom', 'sourceNotes']

saveFile('recipes', output, dictName)

get Recipes 609
get None 5845
Obtained during the main storyline during the villager house development quest if the native fruit is apples
Purchase Test Your DIY Skills from Nook's Cranny; Obtained during the main storyline during the villager house development quest
Obtained during the main storyline during the villager house development quest if the native flower is roses
Obtained during the main storyline during the villager house development quest if the native flower is pansies
Obtained during the main storyline during the villager house development quest if the native fruit is oranges
Obtained during the main storyline during the villager house development quest if the native flower is mums
Obtained during the main storyline during the villager house development quest if the native fruit is cherries
Obtained during the main storyline during the villager house development quest if the native flower is windflowers
50% chance of being obtained during the main storyline during the vi

In [ ]:
output = []

villagers = getItem('Villagers')

for item in villagers:
    if item['category'] != "Villagers":
        continue

#     print(item['content'])
    pickData = {'type': 'villagers'}
    name = en2twFn(item['name'][0].upper() + item['name'][1:], 'onlySite')
#     print(item['name'], name['zh-tw'])

    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['filename'] = item['content']['filename']
    pickData['birthday'] = item['content']['birthday']
    pickData['catchphrase'] = en2twFn(item['content']['catchphrase'])['zh-tw']
    pickData['colors'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['colors']]
    pickData['favoriteSong'] = en2twFn(item['content']['favoriteSong'])['zh-tw']
    pickData['gender'] = en2twFn(item['content']['gender'], True)['zh-tw']
    
    pickData['hobby'] = en2twFn(item['content']['hobby'], True)['zh-tw']
    pickData['personality'] = en2twFn(item['content']['personality'], True)['zh-tw']

    pickData['species'] = en2twFn(item['content']['species'], True)['zh-tw']

    pickData['houseImage'] = item['content']['houseImage'].split('/')[-1].replace('.png', '')
    
    pickData['wallpaper'] = {
        'name': en2twFn(item['content']['wallpaper'])['zh-tw'], 
        'filename': name2fileNameMap[item['content']['wallpaper'][0].upper() + item['content']['wallpaper'][1:]]
    }
    pickData['flooring'] = {
        'name': en2twFn(item['content']['flooring'])['zh-tw'], 
        'filename': name2fileNameMap[item['content']['flooring'][0].upper() + item['content']['flooring'][1:]]
    }
    pickData['furnitureList'] = [idMap[int(furniture)] for furniture in item['content']['furnitureList']]
    
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'filename', 'birthday', 'catchphrase', 'colors', 'favoriteSong']
dictName += ['gender', 'hobby', 'personality', 'species', 'houseImage', 'wallpaper', 'flooring', 'furnitureList']

saveFile('villagers', output, dictName)

In [45]:
jpNameMap = {}
url = 'https://gamepedia.jp/ac-switch/archives/28527'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

table = soup.find("table", class_="data-table table-datatable locale-compare-table")
# print(table)

# print(table.prettify())

for tr in table.find("tbody").find_all("tr"):
    tds = tr.find_all('td')
    name = tds[0].find('div').string

    img = tds[1].find('img')
    imgSrc = None
    if img:
        imgSrc = img.get('src').split('?')[0]
    
#     print(name, imgSrc)
    jpNameMap[name.replace('1', '１').replace('2', '２').replace('3', '３').replace('4', '４')] = imgSrc

print('get {} jp name'.format(len(jpNameMap.keys())))

get 391 jp name


In [48]:
villagers = getItem('Villagers')
limit = 0

# https://gamepedia.jp/ac-switch/archives/28527
# https://wiki.biligame.com/dongsen/%E9%98%B3%E6%98%8E

for item in villagers:
    if item['category'] != "Villagers":
        continue
    
#     limit += 1
#     if limit > 10:
#         break
    
    name = en2twFn(item['name'][0].upper() + item['name'][1:], 'onlySite')
#     print(name['zh-tw'])
    
    name['zh-cn']
    imageUrl_jp = jpNameMap[name['ja-jp']]

    wikiUrl = 'https://animalcrossing.fandom.com/wiki/{}'.format(name['en-us'])
    
    r = requests.get(wikiUrl)
    text = r.text
    
    if imageUrl_jp:
        r_ = requests.get(imageUrl_jp, stream=True)
        if r_.status_code == 200:
            with open('../img/villager/{}_interior.png'.format(name['en-us']), 'wb') as f:
                for chunk in r_:
                    f.write(chunk)
    else:
        m = re.search('https://vignette\.wikia\.nocookie\.net/animalcrossing/images/.*\n.*\n.*\n.*\n<td><i><a href=\"/wiki/Animal_Crossing:_New_Horizons\" title=\"Animal Crossing: New Horizons\">New Horizons</a></i> \(interior\)', text)
        if m:
            mm = re.search('https://vignette\.wikia\.nocookie\.net/animalcrossing/images/.*?g/', m.group())
            if mm:
                interiorUrl = mm.group()[:-1]
                r_ = requests.get(interiorUrl, stream=True)
                if r_.status_code == 200:
                    with open('../img/villager/{}_interior.png'.format(name['en-us']), 'wb') as f:
                        for chunk in r_:
                            f.write(chunk)
        else:
            print('-------{} lose house image'.format(name['zh-tw']))
    
    
    m = re.search('<a href=\"(.*.png).*\n.*\n.*\n.*\n.*\n.*\n.*<figcaption class=\"pi-item-spacing pi-caption\">.*</figcaption>', text)
    if m:
        mm = re.search('https://vignette\.wikia\.nocookie\.net/animalcrossing/images/.*.png', m.group())
        if mm:
            imageUrl = mm.group()
            
            r_ = requests.get(imageUrl, stream=True)
            if r_.status_code == 200:
                with open('../img/villager/{}.png'.format(name['en-us']), 'wb') as f:
                    for chunk in r_:
                        f.write(chunk)
    else:
        print('-------{} lose body image'.format(name['zh-tw']))

get Villagers 1234
-------艾栗絲 lose house image
-------艾栗絲 lose body image
-------蘋果 lose body image
-------貴妃 lose house image
-------貴妃 lose body image
-------花娜 lose body image
-------朱古莉 lose house image
-------太平 lose house image
-------歐若拉 lose body image
-------巧蔻 lose body image
-------阿妹 lose body image
-------鳳尾 lose body image
-------黃金雞 lose house image
-------拉拉米 lose body image
-------李克 lose house image
-------甘油 lose body image
-------小嵐 lose house image
-------小啾 lose house image


In [19]:
output = []
reactions = getItem('Reactions')

for item in reactions:

#     print(item['name'])
#     print('')
    pickData = {'type': 'reactions'}
    name = en2twFn(item['name'])

    pickData['name_c'] = name['zh-tw']
    pickData['filename'] = item['content']['internalID']
    pickData['obtainedFrom'] = ''.join([en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['obtainedFrom']])

    output.append(pickData)

# output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'filename', 'obtainedFrom']

saveFile('reactions', output, dictName)

get Reactions 44
save reactions over


In [10]:
en2twFn = getEn2twFn()
output = []
clothes = getItem('Clothing')

# themes = set()

for item in clothes:
    pickData = {'type': 'clothes'}
    name = en2twFn(item['name'])
    
    print(name['zh-tw'])
    
    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
    pickData['name_e'] = name['en-us'] if 'en-us' in name else ''
    
    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['filename'] = item['content']['filename']
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['sell'] = item['content']['sell']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else ''
    
    pickData['seasonalAvailability'] = en2twFn(item['content']['seasonalAvailability'])['zh-tw']
    pickData['styles'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['styles']])
    pickData['themes'] = [en2twFn(item)['zh-tw'] for item in item['content']['themes']]
    pickData['villagerEquippable'] = item['content']['villagerEquippable']
    
    if 'variations' in item:
        pickData['variations'] = len(item['variations'])
    else:
        pickData['variations'] = 0
        
        
#     for t in item['content']['themes']:
#         themes.add(t)

    output.append(pickData)
    
output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'name_j', 'name_e', 'category', 'filename', 'obtainedFrom', 'sell', 'buy', 'seasonalAvailability', 'styles', 'themes', 'villagerEquippable', 'variations']

saveFile('clothes', output, dictName)

# for t in themes:
#     print(t)

get 6047 i18n data
Custom 1200 i18n data
get Clothing 1161
皮裙
青蛙頭套
橫條紋T恤
無袖有領襯衫
菱格紋毛衣
盛裝和服
摩斯大衣
工作服
MA-1防風裙
長版襯衫
花紋圖案細肩帶背心
綿羊頭套
騎士皮外套
美式足球安全帽
妖精服
凱絲帽子
大圍巾T恤
漆彈面罩
泰拳褲
絲質小禮帽
七分褲
荷葉邊棉褲
餐館制服
鮮艷髮飾
籃球衣
燈芯絨裙
運動褲
星星髮飾
貴族大衣
風衣
骷髏T恤
雙樑眼鏡
無肩帶背心
蓬蓬裙
樂福鞋
圍裙連袖套
花紋T恤
針織帽
龐克裙褲
運動涼鞋
無袖連帽上衣
籃球鞋
油布外套
連身工作服搭袖套
琥珀眼鏡
絨毛外套
扣帶鞋
刷毛外套
嘻哈風衣服
檸檬裙子
拼布大衣
夢幻色系格子連身裙
扣帶低跟鞋
號碼衣
西裝裙
生日墨鏡
牛仔吊帶褲
植物圖案T恤
阿拉伯服
馬德拉斯格紋襯衫
絨面靴
喇叭褲
荷葉邊褲子
民族風刺繡罩衫
雪花毛衣
晚禮服洋裝
保齡球衫
瓜地馬拉傳統裙子
綿羊服
鬥牛犬運動服
下雪毛衣
針織褲
針織裙
商務皮鞋
迷你墨鏡
柔道服
毛球報童帽
學生帽
咖啡店制服
連帽連身裙
清爽五分褲
韓服裙
高筒運動鞋
皮風衣
迷彩衣服
軍師服
雪國服
單車帽
籃球褲
長版襯衫連身裙
針織罩衫&襯衫
彩虹針織衫
學院風針織罩衫
花紋裙子
公主假髮
亮片刺繡無袖上衣
護士帽
健身坦克背心
西裝外套
安全帽
橡膠圍裙
馬丁靴
動力頭盔
亮面西裝褲
肌肉裝
尖頭靴
拼布鬱金香帽
美式足球褲
絨毛連帽上衣
農作遮陽帽
運動短褲
騎士衣服
阿拉伯公主服
全身緊身衣
網帽
格紋圍巾配毛衣
寬邊草帽
羅馬涼鞋
雪國洋裝
羊毛混紡牛角釦大衣
貴族風洋裝
廚師衣服
動物拖鞋
搞笑藝人亮片褲
花紋蕾絲裙
莫卡辛靴
粗橫紋T恤
步行鞋
簡樸衣服
凱絲五分褲
水手襯衫
凱絲低跟鞋
雕花紳士鞋
麂皮運動鞋
連帽上衣配外套
交叉V領綁帶連身裙
翼紋紳士鞋
尖頭鞋
木鞋
制服褲
Polo衫
法蘭絨格紋熱褲
水涼鞋
奇馬約圖騰背心
蕾絲連身裙
男性簡便和服
貓咪服
室內拖鞋
粗橫紋坦克背心
忍者服
麂皮雪靴
騎師服
護士連身裙
原始人服
摩登少女連身裙
格紋背心裙
毛毛靴
巫女服
圍裙連裙子
動物花紋裙
莫卡辛鞋
迷彩裙
無領襯衫
交叉帶涼鞋
蒸汽龐克護目鏡
蕾絲裙
凱絲運動鞋
毛球毛衣
荷蘭洋裝
撞色有領襯衫
豹紋低跟

In [12]:
output = []
clothes = getItem('Home%20Style')

for item in clothes:
    pickData = {'type': 'homeStyle'}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
    pickData['name_e'] = name['en-us'] if 'en-us' in name else ''
    
    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['filename'] = item['content']['filename']
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['sell'] = item['content']['sell']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else ''
    pickData['DIY'] = item['content']['dIY']
    
    pickData['tag'] = en2twFn(item['content']['tag'])['zh-tw']
    pickData['themes'] = [en2twFn(theme)['zh-tw'] for theme in item['content']['themes'] if not theme in ['None', 'none']]
    
    if len(item['variations']) > 0:
        print(name['zh-tw'])
    
    output.append(pickData)
    
output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'name_j', 'name_e', 'category', 'filename', 'obtainedFrom', 'sell', 'buy', 'tag', 'themes']

saveFile('homeStyle', output, dictName)

# for t in themes:
#     print(t)

get Home%20Style 561
save homeStyle over


In [14]:
output = []
equippables = getItem('Equippables')

for item in equippables:
    pickData = {'type': 'equippables'}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
    pickData['name_e'] = name['en-us'] if 'en-us' in name else ''
    
    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['filename'] = item['content']['filename']
    pickData['obtainedFrom'] = [en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']]
    pickData['sell'] = item['content']['sell']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else ''
    
    pickData['DIY'] = item['content']['dIY']
    
    if pickData['category'] == '工具' and pickData['name_c'][-1] != '棒':
        continue

    output.append(pickData)
    
output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'name_j', 'name_e', 'category', 'filename', 'obtainedFrom', 'sell', 'buy']

saveFile('equippables', output, dictName)

get Equippables 131
save equippables over


In [25]:
outputFish = []
outputInsects = []
equippables = getItem('Museum')

for item in equippables:
    
    if not item['category'] in ["Fish", "Insects"]:
        continue
    
    if item['category'] == "Fish":
        pickData = {'type': item['category'].lower()}
        name = en2twFn(item['name'])

        pickData['id'] = int(item['content']['critterId'])
        pickData['name_c'] = name['zh-tw']
        pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
        pickData['name_e'] = name['en-us'] if 'en-us' in name else ''

        pickData['category'] = item['content']['category']
        pickData['filename'] = item['content']['iconFilename']
        pickData['sell'] = item['content']['sell']
        pickData['obtainedFrom'] = item['content']['obtainedFrom']
        pickData['weather'] = item['content']['increasedWeatherChance']
        
        pickData['shadow'] = item['content']['shadow']
        pickData['activeMonths'] = item['content']['activeMonths']
        pickData['catchUnlockCount'] = item['content']['catchUnlockCount']
        pickData['spawnRates'] = item['content']['spawnRates']

        outputFish.append(pickData)

    if item['category'] == "Insects":
        pickData = {'type': item['category'].lower()}
        name = en2twFn(item['name'])

        pickData['id'] = int(item['content']['critterId'])
        pickData['name_c'] = name['zh-tw']
        pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
        pickData['name_e'] = name['en-us'] if 'en-us' in name else ''

        pickData['category'] = item['content']['category']
        pickData['filename'] = item['content']['iconFilename']
        pickData['sell'] = item['content']['sell']
        pickData['obtainedFrom'] = item['content']['obtainedFrom']
        pickData['weather'] = item['content']['weather']
        
        pickData['activeMonths'] = item['content']['activeMonths']
        pickData['catchUnlockCount'] = item['content']['catchUnlockCount']
        pickData['spawnRates'] = item['content']['spawnRates']

        outputInsects.append(pickData)

outputFish = sorted(outputFish, key=lambda k: k['id'])
dictName = ['id', 'name_c', 'name_j', 'name_e', 'category', 'filename', 'sell', 'obtainedFrom', 'weather', 'shadow', 'catchUnlockCount', 'spawnRates', 'activeMonths']

saveFile('fish', outputFish, dictName)

outputInsects = sorted(outputInsects, key=lambda k: k['id'])
dictName = ['id', 'name_c', 'name_j', 'name_e', 'category', 'filename', 'sell', 'obtainedFrom', 'weather', 'catchUnlockCount', 'spawnRates', 'activeMonths']

saveFile('insects', outputInsects, dictName)

get Museum 303
save fish over
save insects over
